# The Han–Monsky ring and the Han–Monsky algorithm
The following cell contains functions that execute the Han–Monsky algorithm for any given diagonal hypersurface.

## What it does:
- Computes the Hilbert–Kunz multiplicity of any diagonal hypersurface
- Computes the Hilbert–Kunz function* of any diagonal hypersurface (see disclaimer below).
- Computes the matrices of the $\delta$-basis and $\lambda$-basis of $\Lambda_e$.

## What it does not do (yet):
- It usually does not work if the characteristic or the exponents of the diagonal hypersurface are large.
- *: The output of the function is $\text{HK}_{\mu e}(R)$ for every $e\geq e_0$, for $\mu$ and $e_0$ certain non-negative integers. The program looks for the minimal possible values for these integers, and in basic examples, $\mu=1$ and $e_0=0$, but this might not be true in general. This will be added later.
- In other words, this function does not compute every $\Delta_e$ for every $e_0<e<e_0+\mu$, but just $\Delta_{e_0}$.

# Instructions:
Very simple: execute the first cell, which will output the Hilbert–Kunz function of
$$
\frac{\mathbb{F}_5[[x_0,x_1,x_2,x_3]]}{(x_0^4+x_1^4+x_2^4+x_3^4)}
$$
as an example.

Then, simply write HK_diagonal(p,[e_0,...,e_d]) in any cell and run it to compute the Hilbert–Kunz function of $x_0^{e_0}+\dots+x_d^{e_d}$ in characteristic $p$. eHK_diagonal(p,[e_0,...,e_d]) computes the Hilbert–Kunz function. HK_diagonal(p,[e_0,...,e_d],vrbs = True) prints out more information in case you want to check by hand.

In [9]:
def lam(p,k):
    a = (p-1)/2
    if p%2 == 0 or k>p-1:
        return 'The first argument needs to be odd and the second must be strictly smaller.'
    l = []
    for i in range(p):
        for j in range(p):
            if j>=i and i+j<=p-1:
                i0 = i
                j0 = j
                x = lambda k: 1 if j0-i0<=k<=i0+j0 else 0
                lij = [x(k) for k in range(p)]
                l.append(lij)
            elif j-i >= 0:
                i0 = p-1-j
                j0 = p-1-i
                x = lambda k: 1 if j0-i0<=k<=i0+j0 else 0
                lij = [x(k) for k in range(p)]
                l.append(lij)
            else:
                j0 = i
                i0 = j
                if i0+j0<=p-1:
                    x = lambda k: 1 if j0-i0<=k<=i0+j0 else 0
                    lij = [x(k) for k in range(p)]
                    l.append(lij)
                else:
                    i1 = p-1-j0
                    j1 = p-1-i0
                    x = lambda k: 1 if j1-i1<=k<=i1+j1 else 0
                    lij = [x(k) for k in range(p)]
                    l.append(lij)
    l = [matrix([l[i+p*k] for i in range(p)]).transpose() for k in range(p)]
    return l[k]


def theta(p,M0):
    '''
    This function takes lambda_k in Lambda_n (as the matrix M0) and outputs the matrix of theta(lambda_k)
    in Lambda_n+1.
    '''
    n = int(log(M0.nrows())/log(p))
    error = 0
    k = list(M0[0]).index(1) #This is to detect which lambda the matrix M0 is representing. But this is easy.
    # It is just the location of the 1 in the first row/column. That's what this line does.
    # This is important to check parity on the next line:
    if k%2!=0: error = 1 # This is for parity.
    M = zero_matrix(p**(n+1),p**(n+1)) # I wouldn't say that these matrices are sparse...
    for a in range(p):
        for i in range(p**n): # We first give the values at elements lambda_ip
            for j in range(p**n):
                # lambda_kp lambda_ip = lambda_error_k * lambda_error_i * theta(lambda_k * lambda_i) =
                # = lambda_error_k * lambda_error_i * sum theta(lambda_j) 
                # = lambda_error_k * lambda_error_i * sum lambda_(pj+error_j)
                if i%2 == 0:
                    if j%2 == 0:
                        M[p*j+error*(p-1)+(-1)**error*a,p*i+a] = M0[j,i]
                    else:
                        M[p*j+p-1-error*(p-1)+(-1)**(1-error)*a,p*i+a] = M0[j,i]
                else:
                    # i is odd now...
                    if j%2 == 0:
                        M[p*j+p-1-error*(p-1)+(-1)**(1-error)*a,p*i+a] = M0[j,i]
                    else:
                        M[p*j+error*(p-1)+(-1)**error*a,p*i+a] = M0[j,i]
    return M

def scaler(p,M0):
    # This function takes the matrix of lambda_k in Lambda_n and gives you the matrix of lambda_k in Lambda_n+1
    return identity_matrix(p).tensor_product(M0)

def p_adic(p,n,k): # digits in base p as list [a_0,a_1,...] for a_0+a_1p+...
    k_p = []
    while k > 0:
        k_p.append(k%p)
        k = k//p
    if n>len(k_p):
        k_p = k_p + [0 for _ in range(n-len(k_p))]
    return k_p

def lambda_matrix(p,n,k):
    if k>=p**n:
        return 'lambda_'+str(k)+'is not in Lambda_'+str(n)
    p_ad = p_adic(p,n,k)
    M = identity_matrix(p**n)
    for i in range(n): #len(p_ad) should be n!!
        Mi = lam(p,p_ad[i])
        for _ in range(i):
            Mi = theta(p,Mi)
        for _ in range(n-i-1):
            Mi = scaler(p,Mi)
        M = M*Mi
    return M

def vertical_swap(M):
    a = M.nrows()
    return matrix([[M[a-j-1,i] for i in range(a)] for j in range(a)])

def N_a(p,n):
    a = int((p**n-1)/2)
    return sum((-1)**k*2*lambda_matrix(p,n,k) for k in range(a))+(-1)**a*lambda_matrix(p,n,a)

def find_mu_e0(p,exps,vrbs = False):
    import itertools
    '''
    The Han–Monsky's algorithm takes the exponents of a diagonal hypersurface, the characteristic,
    and a certain mu. First, it modifies beta so that the denominators are not divisible by it.
    Then, you can start looking for mu. If the correct mu is chosen, then the algorithm yields 
    an expression for the Hilbert–Kunz function of the diagonal hypersurface that works for every 
    e≥e_0 for a certain e_0.
    
    If the hypersurface is an A_n singularity, it can be seen that n+1 = c*p**e0 where gcd(c,p) = 1.
    
    This function computes the least mu such that p**mu = ± 1 (mod c). (see [HM93], Theorem 5.2)

    In any case, this algorithm below is really about trying one by one the values.

    There is an upper bound (max_mu below), and my guess is that all the mu's conputed by this program are divisors of that.
    However, it does not feel that this part of the program is going to slow down things.
    '''
    beta = vector([1/x for x in exps])
    # first, the function computes the minimal e0. this is the first obstruction to speed that the han-monsky algorithm has
    e0 = 0
    for x in exps:
        while x % p^(e0+1) == 0:
            e0 += 1
    newbeta = p^e0*beta
    fracs = [(numerator(x),denominator(x)) for x in newbeta]
    r = vector([int(x) for x in newbeta])
    # now, the function finds the minimal mu for the han-monsky algorithm. this uses the remark in section 5: tries the mu one by
    # one, checking whether z_prime, which is a vector that depends on mu, is equivalent to z or z_star
    mu = 1
    max_mu = lcm([euler_phi(x[1]) for x in fracs])
    if vrbs: print('Maximum mu:',max_mu)
    # the goal now is find the smallest mu such that z' (which depends on z') is equivalent to either z or z*
    units = list(itertools.product([1,-1],repeat=len(exps)))
    class_z = [[u[i]*fracs[i][0]%fracs[i][1] for i in range(len(exps))] for u in units if prod(u)==1]
    class_z_star = [[u[i]*fracs[i][0]%fracs[i][1] for i in range(len(exps))] for u in units if prod(u)==-1]
    z_prime = [p^mu*x[0]%x[1] for x in fracs]
    R = [int(x) for x in p^mu*newbeta]
    z_sim_z_prime = bool(z_prime in class_z)
    z_star_sim_z_prime = bool(z_prime in class_z_star)
    condition = bool(sum(R) % 2 == sum(r) % 2)
    while not (z_sim_z_prime and condition) and not (z_star_sim_z_prime and not condition) and mu<max_mu:
        mu +=1
        z_prime = [p^mu*x[0]%x[1] for x in fracs]
        R = [int(x) for x in p^mu*newbeta]
        z_sim_z_prime = bool(z_prime in class_z)
        z_star_sim_z_prime = bool(z_prime in class_z_star)
        condition = bool(sum(R) % 2 == sum(r) % 2)
        if vrbs: print('z_prime is',z_prime,'for',mu,'and z~z_prime is',z_sim_z_prime,'; z_star~z_prime is',z_star_sim_z_prime,'; condition is',condition,'and sumR',sum(R))
    if vrbs: print('sum(R_i) has the same parity as sum(r_i)',condition,'\nz is equiv to z_prime:',z_sim_z_prime,'\nz_star is equiv to z_prime:',z_star_sim_z_prime)
    return mu,e0,newbeta
    
def eHK_diagonal(p,exps,vrbs = False,HK_function=False):
    '''
    This program computes the Hilbert–Kunz multiplicity of a diagonal hypersurface via the Han–Monsky algorithm.

    It needs to compute three numbers in order to compute it: two specific values of the HK function, and the number lsh.
    The number lsh has to be computed using matrices (or a recursive formula given by Han and Monsky, but we use matrices),
    and the two values of HK function that are needed are here computed by brute force. They can be computed via matrices,
    but there does not seem to be any advantage to that, given that the matrices get very big very quickly.

    When the HK_function option is turned on, the function returns the formula for the HK function of the singularity,
    but maybe not the complete function. If mu = 1, then the function works for every e, but if mu is more than one,
    then one has to compute several functions, one for each value smaller than mu. Thus, the function only gives one set
    of these values, which is expressed as, for example: HK_(3e) = ... because the identity for HK_e only works for multiples
    of 3. So, be careful with this. However, it is not very difficult to modify the function to compute the rest of the parts,
    if you are really interested in that.
    '''
    if vrbs: print('Original beta:',vector([1/e for e in exps]))
    mu,e0,beta = find_mu_e0(p,exps)
    if vrbs: print('data and new beta:','mu is',mu,', e0 is',e0,'and the new beta is',beta)
    dim = len(exps)-1
    r = vector([floor(x) for x in beta])
    z = beta - r
    t = vector([floor(x) for x in p**mu*z])
    z_prime = p**mu*z - t
    if vrbs: print('sum(r) is odd:',sum(r) % 2 != 0,'because sum(r) is',sum(r))
    if sum(r) % 2 != 0:
        t[0] = p^mu-1-t[0]
    # we have to produce l#, and that REQUIRES MATRICES ANYWAY
    lsh = (prod(lambda_matrix(p,mu,ti) for ti in t))[0,0]
    if vrbs: print('l_sharp is:',lsh)
    vars = var(['x%d'%i for i in range(len(exps))])
    R = PolynomialRing(GF(p),vars)
    Ie0 = R.ideal([sum(var('x%d'%i)^exps[i] for i in range(len(exps)))]+[var('x%d'%i)^(p^e0) for i in range(len(exps))])
    Ie0mu = R.ideal([sum(var('x%d'%i)^exps[i] for i in range(len(exps)))]+[var('x%d'%i)^(p^(e0+mu)) for i in range(len(exps))])
    HKe0 = Ie0.vector_space_dimension()
    HKe0mu = Ie0mu.vector_space_dimension()
    var('x y')
    if vrbs: print('The result is given by the solution of the easy linear system of equations:',[HKe0 == x*p^(dim*e0)-y,HKe0mu == x*p^(dim*(e0+mu))-y*lsh])
    if not HK_function:
        return (HKe0mu-HKe0*lsh)/((p**(mu*dim)-lsh)*p**(e0*dim))
    else:
        eHK = (HKe0mu-HKe0*lsh)/((p**(mu*dim)-lsh)*p**(e0*dim))
        var('p,e')
        return 'HK_('+str(e*mu)+')='+str(eHK*p^(dim*(mu*e)))+str(-(eHK-1)*lsh^e)+' for e>='+str(e0)
def HK_diagonal(p,exps,vrbs = False):
    return eHK_diagonal(p,exps,vrbs,HK_function = True)
HK_diagonal(5,[4,4,4,4])

'HK_(e)=168/61*5^(3*e)-107/61*3^e for e>=0'

In [12]:
HK_diagonal(5,[3,3,3]),HK_diagonal(7,[3,3,3]),HK_diagonal(11,[3,3,3])

('HK_(e)=9/4*5^(2*e)-5/4 for e>=0',
 'HK_(e)=9/4*7^(2*e)-5/4 for e>=0',
 'HK_(e)=9/4*11^(2*e)-5/4 for e>=0')